<a href="https://colab.research.google.com/github/zawuya/Dlab-Project/blob/master/imbalancedata1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install -U imbalanced-learn
! pip install -U scikit-learn

In [ ]:
import pandas as pd
import numpy as np
import re
import seaborn as sns
import matplotlib.pyplot as plt
from imblearn.under_sampling import RandomUnderSampler
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
#from sklearn.cross_validation import train_test_split
import joblib # saving models
import os #creating folders

from collections import Counter #counting data
import collections
from imblearn.over_sampling import SMOTE #Balancing data

import imblearn
#Split arrays or matrices into random train and test subsets

# import library
from imblearn.under_sampling import RandomUnderSampler

# import library
from imblearn.over_sampling import RandomOverSampler


import pandas as pd
import numpy as np
import seaborn as sns
import re 
import os
import joblib 
import matplotlib.pyplot as plt
import warnings
import xgboost as xgb
#import catboost as ctb
import lightgbm as ltb
import collections
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay 
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from xgboost import plot_importance
from xgboost import XGBClassifier
#from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler
from sklearn.preprocessing import LabelEncoder

In [ ]:
data=pd.read_csv("telecom_TZ_sentiment_data2.csv", dtype=str)
data

In [ ]:
#removing irrelevant value "complains"
data.drop(data.loc[data['sentiment']=='complains'].index, inplace=True)

In [ ]:
# drop duplicated data
data.drop_duplicates(subset=['comment'], keep='first', inplace=True)

In [ ]:
# drop nullvalues from the dataset
data.dropna(subset=['sentiment', 'comment'], inplace=True)

In [ ]:
data.count()

In [ ]:
# This is a list of swahili stopwords used in the process
swahili_stopwords = [
    'akasema', 'alikuwa', 'alisema', 'baada', 'basi', 'bila', 'cha', 'chini',
    'hadi', 'hapo', 'hata', 'hivyo', 'hiyo', 'huku', 'huo', 'ila', 'ili',
    'ilikuwa', 'juu', 'kama', 'karibu', 'katika', 'kila', 'kima', 'kisha',
    'kubwa', 'kutoka', 'kuwa', 'kwa', 'kwamba', 'kwenda', 'kwenye', 'la',
    'lakini', 'mara', 'mdogo', 'mimi', 'mkubwa', 'mmoja', 'moja', 'muda',
    'mwenye', 'na', 'naye', 'ndani', 'ni', 'nini', 'nonkungu', 'pamoja', 'pia',
    'sana', 'sasa', 'sauti', 'tafadhali', 'tena', 'tu', 'vile', 'wa', 'wakati',
    'wake', 'walikuwa', 'wao', 'watu', 'wengine', 'wote', 'ya', 'hivi', 'huu',
    'hii', 'yake', 'yangu', 'yao', 'yeye', 'yule', 'vya', 'za', 't', 'co',
    'tz', 'au', 'tanzania', 'zaidi', 'zake', 'si','mm','ndo','hapa','je','hawa','nyie'
]

In [ ]:
def clean_data_set(list_text):
    new_list = []
    stopwords_df = pd.read_csv("swahili_stopword.csv", dtype=str, low_memory=False, encoding="utf-8")
    #stopwords_df = swahili_stopwords
    stopwords_list = list(stopwords_df['StopWords'])
    
    for x in list_text:
        x = re.sub(r'@[\w]+','', str(x)) # remove twitter handle
        x = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', str(x)) # removes Url
        x = re.sub(r'[^\w\s]',' ', str(x))    # Remove Panctuations /?!. 
        x = x.strip()                   # Remove leading and trailing spaces
        x = re.sub(' +', ' ', x)        # Remove extra white spaces
        x = re.sub('[^A-Za-z0-9]+', ' ', x)  # Remove special characters
        x = x.lower()                   # Converts to lower case
        x = ' '.join([word for word in x.split() if word not in stopwords_list]) # Removes stopwords
        new_list.append(x)
    return new_list

In [ ]:
data["clean_comment"] = clean_data_set(data.comment.to_list())
# assigning new value to hold clean comments obtained from the function

In [ ]:
data.to_csv("clean_telecom_data.csv", index=False)

In [ ]:
data = pd.read_csv('clean_telecom_data.csv')
data

In [ ]:
# Assign for data inputs and target set
x=data['clean_comment']
y=data['sentiment']

In [ ]:
y

In [ ]:
#dataset features
x

In [ ]:
#target class distribution
y.value_counts()

In [ ]:
#show pie plot
fig1, ax1 = plt.subplots()
ax1.pie(y.value_counts(), autopct='%.2f', labels=data['sentiment'].value_counts())

In [ ]:
print(sns.displot(y))

In [ ]:
ros = RandomOverSampler(random_state=42)
# fit predictor and target variable
x_ros, y_ros = ros.fit_resample(x, y)

In [ ]:
#show pie plot
fig1, ax1 = plt.subplots()
ax1.pie(y_ros.value_counts(), autopct='%.2f', labels=data['sentiment'].value_counts())

In [ ]:
y_ros.value_counts()

In [ ]:
print('Original dataset before over sampling', Counter(y))
print('Resample dataset after over sampling', Counter(y_ros))

Data Splitting

In [ ]:
Xb_train, Xb_test, yb_train, yb_test = train_test_split(x_ros, y_ros,test_size=0.2, random_state=42, shuffle=True)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x, y,test_size=0.8, random_state=42, shuffle=True)

Converting into series

In [ ]:
Xb_train = Xb_train.squeeze()
Xb_test = Xb_test.squeeze()
yb_train = yb_train.squeeze()
yb_test = yb_test.squeeze()

X_train = X_train.squeeze()
X_test = X_test.squeeze()
y_train = y_train.squeeze()
y_test = y_test.squeeze()
type(X_train)

In [ ]:
type(X_test)

MODELLING

In [ ]:
path = 'models'
isExisting = os.path.exists(path)
if not isExisting:
  os.mkdir(path)

In [ ]:
print("Training sample size = ", Xb_train.shape[0])
print("Testing sample size = ", X_test.shape[0])

SUPPORT VECTOR MACHINE (SVM)

In [ ]:
# Initialize a vectorization and modelling pipeline
svm_pipeline = Pipeline([
     ('vect', CountVectorizer(ngram_range=(1,1))),
     ('tfidf', TfidfTransformer()),
     ('clf', SVC(kernel = 'sigmoid', random_state=0, gamma='scale', C=1.2, probability=True)), #tunned svm
 ])

In [ ]:
%%time
# SVM Algorithm training X_train, X_test, y_train, y_test
svm_model = svm_pipeline.fit(Xb_train, yb_train)
svm_model